In [11]:
#TODO import
import sys
print(sys.path)
sys.path.append('./raft1/core')
sys.path.append(r'E:\course\31\machineLearning\AIforMedicine\re2D3D')
sys.path.append(r'E:\course\31\machineLearning\AIforMedicine\re2D3D')
import torch
import torch.nn as nn
from DIRN import generate_contour,PPCSolver,loss_dirn
from raft1.core.raft import RAFT as raft
from Pointnet_Pointnet2_pytorch.models.pointnet2_cls_msg import get_model
from argparse import Namespace
from Barlow_Twins import wrapper
from fenc import BasicEncoder
from Barlow_Twins.loss import BarlowTwinsLoss as Loss_BT
from torch.utils.data import Dataset
import functools
from CUT.models.patchnce import PatchNCELoss

['E:\\pycharm-pro\\PyCharm 2022.2.3\\plugins\\python\\helpers-pro\\jupyter_debug', 'E:\\pycharm-pro\\PyCharm 2022.2.3\\plugins\\python\\helpers\\pydev', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D', 'F:\\Python\\Python310\\Lib\\site-packages\\Demic-0.1', 'F:\\Python\\Python310\\python310.zip', 'F:\\Python\\Python310\\DLLs', 'F:\\Python\\Python310\\lib', 'F:\\Python\\Python310', '', 'C:\\Users\\HASEE\\AppData\\Roaming\\Python\\Python310\\site-packages', 'F:\\Python\\Python310\\lib\\site-packages', 'F:\\Python\\Python310\\lib\\site-packages\\win32', 'F:\\Python\\Python310\\lib\\site-packages\\win32\\lib', 'F:\\Python\\Python310\\lib\\site-packages\\Pythonwin', './raft/core', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\Pointnet_Pointnet2_pytorch\\log', 'E:\\course\\31\\machineLearning\\AIforMedicine\\re2D3D\\Pointnet_Pointnet2_pytorch\\models', 'E:\\

ImportError: cannot import name 'RAFT' from 'raft1' (unknown location)

In [3]:
!pip install pynrrd

Looking in indexes: http://pypi.douban.com/simple/

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pickle
import os
from sklearn.model_selection import train_test_split
from torch.utils import data
from skimage import io
from cv2.cv2 import resize
import numpy as np
#TODO utils & params including dataloaders,gpus,etc...
device = 'cuda' if torch.cuda.is_available() else 'cpu'
img_pth = r'E:\course\31\machineLearning\AIforMedicine\re2D3D\drr_real\2dImage'
#2DIMAGE:_1:I_r  _2:I_fr   _3:I_m   _4:I_m_sa
Vol_pth = r'E:\course\31\machineLearning\AIforMedicine\re2D3D\drr_real\volume'
label_pth = 'path/to/label'
#3D-CT:切记序号要与2Dimage一一对应

#TODO 这里的标注可能还需要进行进一步的调整 因为一张VOl是对应多张2DImage的
class MyDataset(data.Dataset):
    """
    Dataset class for converting the data into batches.
    The data.Dataset class is a pyTorch class which help
    in speeding up  this process with effective parallelization
    """
    'Characterizes a dataset for PyTorch'

    def __init__(self,list_IDs):
        'Initialization'
        self.list_IDs = list_IDs

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

    def __getitem__(self,index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]
        # Load data and get label
        V =  torch.Tensor(
            io.imread(Vol_pth)
        )
        #TODO 这里需不需要再进行一步处理过程呢 preprocess可不可以直接放在这儿做
        I_r = torch.Tensor(
            resize(io.imread(img_pth + ID + '_1.jpg'), (480, 480, 3)))
        I_fr = torch.Tensor(
            resize(io.imread(img_pth + ID + '_2.jpg'), (480, 480, 3)))
        I_m = torch.Tensor(
            resize(io.imread(img_pth + ID + '_3.jpg'), (480, 480, 3)))
        I_m_sa = torch.Tensor(
            resize(io.imread(img_pth + ID + '_4.jpg'), (480, 480, 3)))
        with open(label_pth+ ID + '_.pickle', 'rb') as f:
            T_gt = pickle.load(f)
        T_gt = torch.Tensor(T_gt)

        return I_r,I_fr,I_m,I_m_sa,V,T_gt

# filename = list(set([x.split('_')[0]
#                          for x in os.listdir(img_pth)]))
# partition ={}
# partition['train'], partition['validation'] = train_test_split(filename, test_size=0.33, random_state=42)
#
# params = {'batch_size': 16,
#           'shuffle': True,
#           'num_workers': 6,
#           'worker_init_fn': np.random.seed(42)
#         }
#
# train_dataset = MyDataset(partition['train'])
# training_generator = data.DataLoader(train_dataset, **params)

In [13]:
#TODO create my dataset

In [15]:

#TODO set config models
"""
input: none
output: nn.module,a modified model from pointnet2
"""

def get_model_pointnet2():
    classifier = nn.Sigmoid()
    model = get_model(1)
    model.fc3 = classifier

    return model

"""
input:args Namespace
output:nn.module,a modified model from raft
"""

args = Namespace(dropout=True, alternate_corr=False,
small=False,mixed_precision=False)

def get_model_raft():
    model = raft(args)
    return model

"""
input:?
output:nn.module,A modified model from BT
"""
params = {

}
def get_model_BT():
    model = wrapper.wrapper()
    return model

class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc, ndf=64, n_layers=1, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator
        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d
        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)
        self.conv = nn.Conv2d(  in_channels=256,
                        out_channels=64,
                        kernel_size=1,
                        stride=1)

    def forward(self, input:torch.Tensor):
        #TODO： We use 1×1 convolution to match the encoded feature map to the input channel dimension of the patch GAN discriminator.
        #input channel: 256*32*32 ---> patch GAN input channel dimension?
        """Standard forward."""
        input =self.conv(input)
        return self.model(input)

        #TODO:pnce loss func:
#TODO 在训练的for循环中要把这个补上 BCE LOSS
def get_model_AFE():#TODO done
    input_nc = 64 # input_nc:num_channels TODO:把这个改成对的
    AFE = NLayerDiscriminator(input_nc)
    return AFE

NameError: name 'Namespace' is not defined

In [16]:
#TODO train CUT

In [17]:
#TODO use CUT to implement domain adaptation
"""
input: Real X-ray image
output: bone-projection style image
"""
def convert_to_drr(I_fr):
    I_f = 0 # TODO 修改这个函数
    return I_f

In [1]:
from torch import Tensor
import random
import numpy as np
import DIRN
"""
TODO 构建一个我自己的model
输入的形式是以一张真实图像一张drr图像，
其中real图像转化为DRR图像的部分我放在一个stand_alone_module
forward而出的是一个Tpred
"""


#TODO how to implement data sharing
def flowmap_to_p1(flowmap,p):#都不是 是一个map的形式
     a,b,c = p.shape
     p1 = np.zeros((a,b,c))
     p = np.nonzero(p)
     for (x,y,z) in p:
        p1[x+flowmap[x,y,z][0],y+flowmap[x,y,z][1],z+flowmap[x,y,z][2]] = 1 # TODO 可以稍微优化一下
     return p1

def concatate(n,p1,w,g):
    #TODO:CONCANATE THEM INTO A MATRIX(how） compute xyz
    xyz = np.concatenate(n,p1,w,g)# TODO 待补充
    return xyz

class MyDIRN(nn.Module):
    def __init__(self):
        self.pointnet = get_model_pointnet2()
        self.raft = get_model_raft()
        self.fenc = BasicEncoder()
    def forward(self,I_m:torch.Tensor,I_fr:torch.Tensor,V):#So我们在做generater的时候一定要做三个都有的dataloader
        w,g,p = generate_contour(V)
        I_f = convert_to_drr(I_fr)

        flowmap = self.raft(I_f,I_m)#这段跑起来的时候可能得要修改一下
        p1 =flowmap_to_p1(flowmap,w,g)
        N_fl = len(flowmap)
        N_cp = len(np.nonzero(p))
        n = np.dot(w,g,p1)
        xyz = concatate(n,p1,w,g)#concanate还没有改

        W = self.pointnet(xyz)
        dv = PPCSolver(p1,w,g,W)

        Tpred = dv

        return Tpred,W,dv,flowmap,N_fl,N_cp

    def loss(self,N_fl,N_cp,Tpred,T_gt,dv,W,f,f_gt,M_p):
        return DIRN.loss_dirn(N_fl,N_cp,Tpred,T_gt,dv,W,f,f_gt,M_p)

class MyBT(nn.Module):
    def __init__(self):
        self.barlowTwins = get_model_BT()
        self.fenc = BasicEncoder()
        self.loss = Loss_BT()
    def forward(self,Im:torch.Tensor,Imsa:torch.Tensor):

        fmap_Im = self.fenc(Im)
        fmap_Imsa = self.fenc(Imsa)
        Z_m = self.barlowTwins(fmap_Im)
        Z_msa = self.barlowTwins(fmap_Imsa)

        return Z_m,Z_msa

    def BTloss(self,Z_m:torch.Tensor,Z_msa:torch.Tensor):
        btloss = self.loss(Z_m,Z_msa)
        return btloss

class MyAFE(nn.Module):
    def __init__(self):
        self.fenc = BasicEncoder()
        self.AFE = get_model_AFE()

    def forward(self,I_fr:torch.Tensor,I_r:torch.Tensor):
        I_f = convert_to_drr(I_fr)
        fmap_I_f = self.fenc(I_f)
        fmap_I_r = self.fenc(I_r)
        random_index = random.randrange(0,1)
        if random_index ==0:
            ran_fmap = fmap_I_f
        else:
            ran_fmap = fmap_I_r
        Ypred = self.AFE(ran_fmap)
        return Ypred

    def loss(self,Y_pred,label):
        return nn.BCELoss(Y_pred,label)


class MyModel(nn.Module):
    def __init__(self):

        self.afe = MyAFE()
        self.bt = MyBT()
        self.DIRN = MyDIRN()

    def forward(self,I_r:torch.Tensor,I_fr:torch.Tensor,I_m:torch.Tensor,I_m_sa:torch.Tensor,V:torch.Tensor):

        Z_m,Z_msa = self.bt(I_m,I_m_sa)
        Tpred,W,dv,flowmap,N_fl,N_cp = self.DIRN(I_m,I_fr,V)
        Ypred = self.afe(I_fr,I_r)

        return Z_m,Z_msa,Tpred,Ypred,W,dv,flowmap,N_fl,N_cp

    def losstotal(self,
                  Z_m,
                  Z_msa,
                  Tpred,
                  W,
                  dv,
                  Ypred,
                  flowmap,
                  N_fl,
                  N_cp,
                  T_gt,
                  f_gt,
                  M_p,
                  label):
        return torch.Tensor(0.05*self.bt(Z_m,Z_msa)+0.2*self.afe(Ypred,label)+self.DIRN.loss(N_fl,N_cp,Tpred,T_gt,dv,W,flowmap,f_gt,M_p))


ModuleNotFoundError: No module named 'update'

In [ ]:
from torch import optim

#TODO train model & save models
#TODO 修改forward，建立网络前后的连接模型  training:是用images进行训练的 eval：
"""
raft:
      input:2*images 640*480(Centercrop) ----> 480 * 480(resized) ------>  imagesize:[256 ,256]
      output:Ir: 1 * fmap [256, 32, 32]  If: 1 * cmap[256,32,32]
      output:Tpred converted from dv size: unknown
pointnet2:
    input: p1,n,w,g
    output:W a diagonal matrix Ncp * Ncp
3d_canny_edge_detection:
    input: V nii.gz
    output: w ,g size:unknown(can be tracked)
deepdrr:
    input : V nii
    output : Idrr (640*480) can be loaded before training
CUT:
    input:Ir & Iadv size:256*256
    output : none loss_fn(what)
Barlow Twins:
    input :1 cmap [256,32,32] 1 fmap [256,32,32]
    output:
PPCsolver:
    input:W,P,P1
    output:Tpred
"""

"""ATTENTION:所有在这里面的数据的格式都是tensor()"""

#TODO：CUDA是怎么实装到上面去的
if torch.cuda.is_available():
   MyModel = MyModel.cuda()
optimizer = torch.optim.Adam(MyModel.parameters(),lr=1e-4)#TODO 名称待补充

epoch = 0
for data in train_loader :
    ID,V,I_r,I_fr,I_m,I_m_sa,T_gt = data
    if torch.cuda.is_available():
        V = V.cuda()
        I_r = I_r.cuda()
        I_fr = I_fr.cuda()
        I_m = I_m.cuda()
        I_m_sa = I_m_sa.cuda()
    Z_m,Z_msa,Tpred,Ypred,W,dv,flowmap,N_fl,N_cp = MyModel(I_r,I_fr,I_m,I_m_sa,V)
    loss = MyModel.losstotal(Z_m,
                  Z_msa,
                  Tpred,
                  W,
                  dv,
                  Ypred,
                  flowmap,
                  N_fl,
                  N_cp,
                  T_gt,
                  f_gt,
                  M_p,
                  label)
    print_loss = loss.data.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch+=1

    if epoch%50 == 0:
        print('epoch: {},loss: {:,4}'.format(epoch,print_loss))




#TODO

In [2]:
import numpy as np
file= r'0.nii.gz'
file[0]

'0'

In [5]:
a = np.zeros((128,256,512))
a.shape[0]

128

In [6]:
np.floor(128/10)

12.0